In [1]:
from pyspark.sql import SparkSession

# generamos contexto spark
spark = (SparkSession.builder
    .appName("Example3")
    .getOrCreate()
)

sc = spark.sparkContext

24/06/19 22:19:51 WARN Utils: Your hostname, codespaces-328e39 resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/06/19 22:19:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/06/19 22:19:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/19 22:19:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
def save_file(rrd, file_path):
    # Generamos nuevo rdd con separador
    rdd_save = rrd.map(lambda x: f"{x[0]};{x[1]}")

    # almacenamos data en path
    rdd_save.saveAsTextFile(file_path)

In [2]:
# 1. lectura de datos
rdd = sc.textFile('./data/casoDePrueba3.txt')

# Convirtiendo datos en tupla
rdd = (rdd
       .map(lambda line: line.split(";"))
       .map(lambda x: (x[0], x[1], float(x[2])))
)

In [5]:
"""
a) Por cada persona indique en cuántas compras pagó más de 1500 euros con un medio de pago
diferente a tarjeta de crédito. La solución se tiene que guardar en un archivo llamado
comprasSinTDCMayorDe1500
"""
# aplicando funcion map según condiciones
rdd_map = rdd.map(lambda x: (x[0], 1 if x[1]!='Tarjeta de crédito' and x[2]>1500 else 0))
rdd_group = rdd_map.reduceByKey(lambda x,y: x+y)

rdd_group.collect()


[('Alice', 0), ('Bob', 1)]

In [7]:
save_file(rdd_group, file_path='./out/comprasSinTDCMayorDe1500')

In [8]:
"""
b)Por cada persona indique en cuántas compras pagó menos o igual a 1500 euros con un medio de
pago diferente a tarjeta de crédito. La solución se tiene que guardar en un archivo llamado
comprasSinTDCMenoroIgualDe1500.
"""

rdd_map = rdd.map(lambda x: (x[0], 1 if x[1]!='Tarjeta de crédito' and x[2]<=1500 else 0))
rdd_group = rdd_map.reduceByKey(lambda x,y: x+y)


In [9]:
save_file(rdd_group, file_path='./out/comprasSinTDCMenoroIgualDe1500')